In [3]:
from transformers import BertConfig, BertForTokenClassification
from transformers import PreTrainedTokenizerFast, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification
from datasets import Dataset
from tokenizers import Tokenizer, models, pre_tokenizers
import torch

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Using device: {device}")

train_data = [
    {"text": "Скидка 20% на все модели Redmi Note 12 Pro до конца недели", "tags": [0,0,0,0,0,1,1,1,1,0,0,0]},
    {"text": "В продажу поступил новый ASUS ROG Strix G17 с процессором AMD", "tags": [0,0,0,0,1,1,1,1,0,0,0]},
    {"text": "Камера Alpha A7 IV от Sony делает потрясающие снимки при слабом освещении", "tags": [0,1,1,1,0,0,0,0,0,0,0,0]},
    {"text": "Band 8 от Huawei держит заряд до 2 недель в режиме ожидания", "tags": [1,1,0,0,0,0,0,0,0,0,0,0]},
    {"text": "Где найти замену аккумулятора для MX Master 3S?", "tags": [0,0,0,0,0,1,1,1]},
    {"text": "Кто-нибудь тестировал Mi Electric Scooter 4 Pro в мороз?", "tags": [0,0,1,1,1,1,1,0,0]},
    {"text": "Apple представила Watch Ultra 2 с новым датчиком", "tags": [0,0,1,1,1,0,0,0]},
    {"text": "Samsung Galaxy Z Flip5 получил обновление камеры", "tags": [1,1,1,1,0,0,0]},
    {"text": "Только что распаковал EOS R6 Mark II - камера просто космос!", "tags": [0,0,0,1,1,1,1,0,0,0,0]},
    {"text": "Где купить PlayStation 5 с гарантией в Москве?", "tags": [0,0,1,1,0,0,0,0]},
    {"text": "Проблемы с драйверами для LaserJet Pro MFP M283fdw на Windows 11", "tags": [0,0,0,0,1,1,1,1,0,0,0]},
    {"text": "GeForce RTX 4090 перегревается в играх", "tags": [1,1,1,0,0,0]},
    {"text": "Для монтажа видео берите Core i9-14900K и ROG Strix Z790-E", "tags": [0,0,0,0,1,1,0,1,1,1]},
    {"text": "SSD 990 Pro 2TB - лучший выбор для игр", "tags": [1,1,1,1,0,0,0,0,0]},
    {"text": "UltraSharp U2723QE от Dell - идеален для цветокоррекции", "tags": [1,1,0,0,0,0,0,0]},
    {"text": "Series 9 от Braun бреет лучше моей старой модели", "tags": [1,1,0,0,0,0,0,0,0]}
]

train_data += [
    {"text": "Купите новый iPhone 15 Pro Max в магазине", "tags": [0, 0, 1, 1, 1, 1, 0, 0]},
    {"text": "Закажите ноутбук Lenovo IdeaPad 5 с доставкой", "tags": [0, 0, 1, 1, 1, 0, 0]},
    {"text": "В продаже появились Samsung Galaxy S24 и Xiaomi 14", "tags": [0, 0, 0, 1, 1, 1, 0, 1, 1]},
    {"text": "Рекомендуем телевизор Sony Bravia XR-55A80J", "tags": [0, 0, 1, 1, 1]},
    {"text": "Попробуйте наушники AirPods Pro 2", "tags": [0, 0, 1, 1, 1]},
    {"text": "Новый пылесос Dyson V15 Absolute", "tags": [0, 0, 1, 1, 1]},
    {"text": "Игра The Last of Us Part II теперь в продаже", "tags": [0, 1, 1, 1, 1, 1, 1, 0, 0, 0]},
    {"text": "Фитнес-браслет Huawei Band 8", "tags": [0, 1, 1, 1]},
    {"text": "Фотоаппарат Canon EOS R6 Mark II", "tags": [0, 1, 1, 1, 1, 1]},
    {"text": "Микроволновка LG MS-2042DB", "tags": [0, 1, 1]},
    {"text": "Смарт-часы Apple Watch Series 9", "tags": [0, 1, 1, 1, 1]},
    {"text": "Электрический чайник Bosch TWK7201", "tags": [0, 0, 1, 1]},
    {"text": "Игровая консоль PlayStation 5 Slim", "tags": [0, 0, 1, 1, 1]},
    {"text": "Монитор Dell UltraSharp U2723QE", "tags": [0, 1, 1, 1]},
    {"text": "Кофеварка DeLonghi Dinamica Plus", "tags": [0, 1, 1, 1]},
    {"text": "Мышь Logitech MX Master 3S", "tags": [0, 1, 1, 1, 1]},
    {"text": "Клавиатура Keychron Q3 Pro", "tags": [0, 1, 1, 1]},
    {"text": "Планшет iPad Air 2024", "tags": [0, 1, 1, 1]},
    {"text": "Электронная книга PocketBook 740", "tags": [0, 0, 1, 1]},
    {"text": "Принтер HP LaserJet Pro MFP M283fdw", "tags": [0, 1, 1, 1, 1, 1]},
    {"text": "Холодильник Bosch Serie 6 VitaFresh", "tags": [0, 1, 1, 1, 1]},
    {"text": "Стиральная машина Samsung WW90TA046AE", "tags": [0, 0, 1, 1]},
    {"text": "Посудомойка Electrolux ESF9453LOX", "tags": [0, 1, 1]},
    {"text": "Робот-пылесос Roborock S8 Pro Ultra", "tags": [0, 1, 1, 1, 1]},
    {"text": "Умная колонка Яндекс Станция 2", "tags": [0, 0, 1, 1, 1]},
    {"text": "Эхолот Garmin Striker Plus 7SV", "tags": [0, 1, 1, 1, 1]},
    {"text": "Дрон DJI Mavic 3 Pro", "tags": [0, 1, 1, 1, 1]},
    {"text": "Электросамокат Xiaomi Mi Electric Scooter 4 Pro", "tags": [0, 1, 1, 1, 1, 1, 1]},
    {"text": "Гироскутер Smart Balance Wheel 10", "tags": [0, 1, 1, 1, 1]},
    {"text": "Электробритва Braun Series 9", "tags": [0, 1, 1, 1]},
    {"text": "Фен Dyson Supersonic HD15", "tags": [0, 1, 1, 1]},
    {"text": "Зубная щетка Oral-B iO Series 9", "tags": [0, 0, 1, 1, 1, 1]},
    {"text": "Весы Withings Body+", "tags": [0, 1, 1]},
    {"text": "Термометр B.Well WF-4000", "tags": [0, 1, 1]},
    {"text": "Геймпад Xbox Wireless Controller", "tags": [0, 1, 1, 1]},
    {"text": "Роутер ASUS RT-AX86U Pro", "tags": [0, 1, 1, 1]},
    {"text": "SSD-накопитель Samsung 990 Pro", "tags": [0, 1, 1, 1]},
    {"text": "Видеокарта NVIDIA GeForce RTX 4090", "tags": [0, 1, 1, 1, 1]},
    {"text": "Процессор Intel Core i9-14900K", "tags": [0, 1, 1, 1]},
    {"text": "Материнская плата ASUS ROG Strix Z790-E", "tags": [0, 0, 1, 1, 1, 1]},
    {"text": "Блок питания Corsair RM1000x", "tags": [0, 0, 1, 1]},
    {"text": "Корпус NZXT H7 Elite", "tags": [0, 1, 1, 1]},
    {"text": "Кулер для процессора Noctua NH-D15", "tags": [0, 0, 0, 1, 1]},
    {"text": "Оперативная память Kingston Fury Beast DDR5", "tags": [0, 0, 1, 1, 1, 1]},
    {"text": "Внешний жесткий диск WD My Passport 5TB", "tags": [0, 0, 0, 1, 1, 1, 1]},
    {"text": "Флешка SanDisk Ultra Fit 256GB", "tags": [0, 1, 1, 1, 1]},
    {"text": "Карта памяти Sony TOUGH 128GB", "tags": [0, 0, 1, 1, 1]},
    {"text": "Кабель USB-C Anker PowerLine III", "tags": [0, 0, 1, 1, 1]},
    {"text": "Док-станция CalDigit TS4", "tags": [0, 1, 1]},
    {"text": "Шуруповерт Makita DDF487SHE", "tags": [0, 1, 1]},
    {"text": "Перфоратор Bosch GBH 2-28", "tags": [0, 1, 1, 1]}
]

# 3. Создание токенизатора (ваш код без изменений)
def build_tokenizer(train_data):
    vocab = set()
    special_tokens = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]
    for item in train_data:
        vocab.update(item["text"].split())
    vocab = {word: idx for idx, word in enumerate(special_tokens + sorted(vocab))}
    tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token="[UNK]"))
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    return PreTrainedTokenizerFast(
        tokenizer_object=tokenizer,
        unk_token="[UNK]",
        pad_token="[PAD]",
        cls_token="[CLS]",
        sep_token="[SEP]",
        mask_token="[MASK]",
    )

tokenizer = build_tokenizer(train_data)

def prepare_dataset(data, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    labels = []
    for item in data:
        tokens = item["text"].split()
        tags = item["tags"]
        encoding = tokenizer(
            tokens,
            is_split_into_words=True,
            truncation=True,
            max_length=max_length,
            padding='max_length',
            return_tensors='pt'
        )
        word_ids = encoding.word_ids()
        aligned_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)
            elif word_id != current_word:
                aligned_labels.append(tags[word_id])
                current_word = word_id
            else:
                aligned_labels.append(tags[word_id] if tags[word_id] != -100 else -100)
        if len(aligned_labels) < max_length:
            aligned_labels += [-100] * (max_length - len(aligned_labels))
        input_ids.append(encoding['input_ids'].squeeze())
        attention_masks.append(encoding['attention_mask'].squeeze())
        labels.append(torch.tensor(aligned_labels))
    return {
        'input_ids': torch.stack(input_ids),
        'attention_mask': torch.stack(attention_masks),
        'labels': torch.stack(labels)
    }

processed_data = prepare_dataset(train_data, tokenizer)
dataset = Dataset.from_dict({
    'input_ids': processed_data['input_ids'].numpy(),
    'attention_mask': processed_data['attention_mask'].numpy(),
    'labels': processed_data['labels'].numpy()
})

config = BertConfig(
    vocab_size=len(tokenizer),
    hidden_size=128,
    num_hidden_layers=4,
    num_attention_heads=4,
    intermediate_size=256,
    max_position_embeddings=512,
    num_labels=2
)
model = BertForTokenClassification(config).to(device)

training_args = TrainingArguments(
    output_dir="./test_ner_model",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    learning_rate=3e-4,
    save_steps=200,
    logging_steps=10,
    report_to="none",
    disable_tqdm=False,
)

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()

def predict_products(text, model, tokenizer, max_length=128):
    tokens = text.split()
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,
        return_tensors="pt",
        truncation=True,
        max_length=max_length,
        padding='max_length'
    ).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)[0].cpu().numpy()
    word_ids = inputs.word_ids()
    products = []
    current_product = []
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            continue
        if predictions[i] == 1:
            current_product.append(tokens[word_id])
        elif current_product:
            products.append(" ".join(current_product))
            current_product = []
    if current_product:
        products.append(" ".join(current_product))
    return products

test_text = "Купите новый iPhone 15 Pro Max и Samsung S24"
print("\n🧪 Тестирование модели:")
print(f"Текст: '{test_text}'")
print("Извлеченные продукты:", predict_products(test_text, model, tokenizer))

Using device: mps


/var/folders/3k/wcc4c2m97_d363_yy26qzp7c0000gn/T/ipykernel_41927/3054360764.py:194: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/oxydear/Documents/Ivan's Mac/IDEs/PyCharm/furnitureRecog/venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss



🧪 Тестирование модели:
Текст: 'Купите новый iPhone 15 Pro Max и Samsung S24'
Извлеченные продукты: ['iPhone 15 Pro Max', 'Samsung S24']
